# Read the infomation from sensors

In [54]:
import sensor

In [57]:
device01 = sensor.device("COM3",9600) #new object from class

In [ ]:
weight,height,temp = device01.getHeightWeightTemp() #retrive weight,height,temp

In [58]:
device01.serialPort.close() #close the connection

In [ ]:
device01.connectAgain() #reset the connection with same parameter

# Playground 

In [34]:
stringstr = "W:50.555     H:166.777"
str = stringstr.find('H')
stringend = stringstr[str+2:len(stringstr)]
end = stringend.find(' ')
if end <= 0 : end = len(stringend)
print(stringend[0:str+end])
#print(end)

166.777


In [5]:
stringstr = "W:50     H:166"
str = stringstr.find('W')
stringend = stringstr[str+2:len(stringstr)]
end = stringend.find(' ')
if end <= 0 : end = len(stringend)
print(stringend[0:str+end])
#print(end)

50


# Read infomation from card

In [58]:
import ID_card
card = ID_card.CardReader() #new object from class


In [72]:
info = card.readCard() #retrive card's infomation and store in variable 'info'

In [35]:
card.checkCard() #check if the card is inserted

0

In [60]:
try:
    while True:
        print(card.cardobserver.CardStatus) #<-- this variable corresponds for if the card is inserted or not
        time.sleep(1)
except KeyboardInterrupt:
    pass

1
1
1
1
0
0
0
0
0
0
1
1


In [66]:
card.cardobserver.CardStatus

0

In [73]:
info

{'CID': '1341501371261',
 'FullnameTH': 'นาย จิระพงษ์  ส่งเสริม',
 'FullnameEN': 'Mr. Jiraphong  Songsoem',
 'Date of birth': '25431105',
 'Gender': '1',
 'Card Issuer': 'อำเภอสำโรง/อุบลราชธานี',
 'Issue Date': '25590323',
 'Expire Date': '25671104',
 'Address': '188 หมู่ที่ 12    ตำบลสำโรง อำเภอสำโรง จังหวัดอุบลราชธานี'}

In [68]:
del info

In [ ]:
info['CID'] #เลขบัตร
info['FullnameTH'] #ชื่อไทย
info['FullnameEN'] #ชื่ออังกฤษ
info['Gender'] #เพศ
info['Card Issuer'] #ผู้ออกบัตร
info['Issue Date'] #วันออก
info['Expire Date'] #วันหมด
info['Address'] #ที่อยู่

# card, sensor and Send infomation 

In [1]:
import connect
import ID_card
import sensor

In [2]:
conn = connect.communication("192.168.67.58","student","stu2022","vitalsign") #new instance of connection

In [3]:
conn.dataBaseConnection.is_connected() #check for connection status

True

In [4]:
device01 = sensor.device("COM3",9600) #new object from class
card = ID_card.CardReader() #new object from class

In [ ]:
conn.sendInfo(CardReader=card,device=device01)

In [6]:
"""
Sample for python PCSC wrapper module: Detect card insertion/removal

__author__ = "http://www.gemalto.com"

Copyright 2001-2012 gemalto
Author: Jean-Daniel Aussel, mailto:jean-daniel.aussel@gemalto.com
Copyright 2010 Ludovic Rousseau
Author: Ludovic Rousseau, mailto:ludovic.rousseau@free.fr

This file is part of pyscard.

pyscard is free software; you can redistribute it and/or modify
it under the terms of the GNU Lesser General Public License as published by
the Free Software Foundation; either version 2.1 of the License, or
(at your option) any later version.

pyscard is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License
along with pyscard; if not, write to the Free Software
Foundation, Inc., 51 Franklin St, Fifth Floor, Boston, MA  02110-1301  USA
"""

from smartcard.scard import *
import smartcard.util

srTreeATR = \
    [0x3B, 0x77, 0x94, 0x00, 0x00, 0x82, 0x30, 0x00, 0x13, 0x6C, 0x9F, 0x22]
srTreeMask = \
    [0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF]


def printstate(state):
    reader, eventstate, atr = state
    print(reader + " " + smartcard.util.toHexString(atr, smartcard.util.HEX))
    if eventstate & SCARD_STATE_ATRMATCH:
        print('\tCard found')
    if eventstate & SCARD_STATE_UNAWARE:
        print('\tState unware')
    if eventstate & SCARD_STATE_IGNORE:
        print('\tIgnore reader')
    if eventstate & SCARD_STATE_UNAVAILABLE:
        print('\tReader unavailable')
    if eventstate & SCARD_STATE_EMPTY:
        print('\tReader empty')
    if eventstate & SCARD_STATE_PRESENT:
        print('\tCard present in reader')
    if eventstate & SCARD_STATE_EXCLUSIVE:
        print('\tCard allocated for exclusive use by another application')
    if eventstate & SCARD_STATE_INUSE:
        print('\tCard in used by another application but can be shared')
    if eventstate & SCARD_STATE_MUTE:
        print('\tCard is mute')
    if eventstate & SCARD_STATE_CHANGED:
        print('\tState changed')
    if eventstate & SCARD_STATE_UNKNOWN:
        print('\tState unknowned')


try:
    hresult, hcontext = SCardEstablishContext(SCARD_SCOPE_USER)
    if hresult != SCARD_S_SUCCESS:
        raise error(
            'Failed to establish context: ' +
            SCardGetErrorMessage(hresult))
    print('Context established!')

    try:
        hresult, readers = SCardListReaders(hcontext, [])
        if hresult != SCARD_S_SUCCESS:
            raise error(
                'Failed to list readers: ' +
                SCardGetErrorMessage(hresult))
        print('PCSC Readers:', readers)

        readerstates = []
        for i in range(len(readers)):
            readerstates += [(readers[i], SCARD_STATE_UNAWARE)]

        print('----- Current reader and card states are: -------')
        hresult, newstates = SCardGetStatusChange(hcontext, 0, readerstates)
        for i in newstates:
            printstate(i)

        print('----- Please insert or remove a card ------------')
        hresult, newstates = SCardGetStatusChange(
                                hcontext,
                                INFINITE,
                                newstates)

        print('----- New reader and card states are: -----------')
        for i in newstates:
            printstate(i)

    finally:
        hresult = SCardReleaseContext(hcontext)
        if hresult != SCARD_S_SUCCESS:
            raise error(
                'Failed to release context: ' +
                SCardGetErrorMessage(hresult))
        print('Released context.')

except error as e:
    print(e)

Context established!
PCSC Readers: ['Generic EMV Smartcard Reader 0']
----- Current reader and card states are: -------
Generic EMV Smartcard Reader 0 
	Reader empty
	State changed
----- Please insert or remove a card ------------
----- New reader and card states are: -----------
Generic EMV Smartcard Reader 0 0x3B 0x79 0x96 0x00 0x00 0x54 0x48 0x20 0x4E 0x49 0x44 0x20 0x31 0x33
	Card present in reader
	State changed
Released context.
press Enter to continue


# Multiprocessing

In [53]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)


p = Process(target=f, args=('bob',))
p.start()
p.join()
